In [1]:
import matplotlib

import matplotlib.pyplot as plt

import numpy as np

import os

import torch

from scipy.io import loadmat

from tqdm import tqdm_notebook as tqdm

In [2]:
%matplotlib inline

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')

In [4]:
# Add new methods here.
# methods = ['ORB', 'ORB+Boost-B', 'SIFT', 'SIFT+Boost-F', 'SIFT+Boost-B', 'RootSIFT', 'SOSNet', 'SuperPoint', 'SuperPoint+Boost-F', 'SuperPoint+Boost-B', 'ALIKE', 'ALIKE+Boost-F', 'ALIKE+Boost-B']
methods = ['SuperPoint', 'SuperPoint+Boost-F', 'SuperPoint+Boost-B']
# names = ['ORB', 'ORB+Boost-B', 'SIFT', 'SIFT+Boost-F', 'SIFT+Boost-B', 'RootSIFT', 'SOSNet', 'SuperPoint', 'SuperPoint+Boost-F', 'SuperPoint+Boost-B', 'ALIKE', 'ALIKE+Boost-F', 'ALIKE+Boost-B']
names = ['SuperPoint', 'SuperPoint+Boost-F', 'SuperPoint+Boost-B']
# colors = ['orange', 'orange', 'green', 'green', 'green', 'blue', 'cyan', 'red', 'red', 'red', 'purple', 'purple', 'purple']
# linestyles = ['-', '--', '-', '--', '-.', '-', '-', '-', '--', '-.', '-', '--', '-.']
colors = ['red', 'red', 'red']
linestyles = ['-', '--', '-.']

In [5]:
n_i = 52
n_v = 56

In [1]:
dataset_path = 'hpatches-sequences-release'
# TODO 数据集路径要改

In [7]:
lim = [1, 15]
rng = np.arange(lim[0], lim[1] + 1)

In [8]:
def mnn_matcher(descriptors_a, descriptors_b):
    device = descriptors_a.device
    sim = descriptors_a @ descriptors_b.t()
    nn12 = torch.max(sim, dim=1)[1]
    nn21 = torch.max(sim, dim=0)[1]
    ids1 = torch.arange(0, sim.shape[0], device=device)
    mask = (ids1 == nn21[nn12])
    matches = torch.stack([ids1[mask], nn12[mask]])
    return matches.t().data.cpu().numpy()

In [9]:
# 这里是评估的主要函数
def benchmark_features(read_feats):
    seq_names = sorted(os.listdir(dataset_path)) # ：这行代码列出了dataset_path目录中的所有文件和文件夹，并将它们排序

    # 初始化了几个列表和字典来存储特征点数量、匹配数量、序列类型以及不同阈值下的错误率和匹配数
    n_feats = []
    n_matches = []
    seq_type = []
    i_err = {thr: 0 for thr in rng}
    i_matches = {thr: 0 for thr in rng}
    v_err = {thr: 0 for thr in rng}
    v_matches = {thr: 0 for thr in rng}

    for seq_idx, seq_name in tqdm(enumerate(seq_names), total=len(seq_names)):
        keypoints_a, descriptors_a = read_feats(seq_name, 1) # 对于每个序列，读取第一幅图像的关键点和描述符
        n_feats.append(keypoints_a.shape[0])

        for im_idx in range(2, 7): # 内部循环，对于每个序列，读取第2到第6幅图像的关键点和描述符
            keypoints_b, descriptors_b = read_feats(seq_name, im_idx)
            n_feats.append(keypoints_b.shape[0])

            matches = mnn_matcher( # 使用最近邻匹配器找到描述符之间的匹配
                torch.from_numpy(descriptors_a).to(device=device), 
                torch.from_numpy(descriptors_b).to(device=device)
            )
            
            homography = np.loadtxt(os.path.join(dataset_path, seq_name, "H_1_" + str(im_idx))) # 加载两幅图像之间的单应性矩阵
            
            # 计算匹配点之间的位置误差
            pos_a = keypoints_a[matches[:, 0], : 2]  # 从关键点数组keypoints_a中提取出所有匹配的第一组关键点的位置信息
            pos_a_h = np.concatenate([pos_a, np.ones([matches.shape[0], 1])], axis=1) # 这里将pos_a的每个点扩展为齐次坐标形式，即在每个点的x和y坐标后面添加了一个1
            pos_b_proj_h = np.transpose(np.dot(homography, np.transpose(pos_a_h))) # 通过单应性矩阵homography将pos_a_h中的点投影到第二幅图像的坐标系中
            pos_b_proj = pos_b_proj_h[:, : 2] / pos_b_proj_h[:, 2 :] # 将投影点的齐次坐标转换回普通坐标

            pos_b = keypoints_b[matches[:, 1], : 2] # 从关键点数组keypoints_b中提取出所有匹配的第二组关键点的位置信息

            dist = np.sqrt(np.sum((pos_b - pos_b_proj) ** 2, axis=1)) # 计算每对匹配点之间的欧氏距离，这是通过取实际点pos_b和投影点pos_b_proj之间差的平方和的平方根来实现的

            n_matches.append(matches.shape[0]) # 将当前匹配的数量添加到n_matches列表中
            seq_type.append(seq_name[0]) # 将当前序列的类型（例如，如果序列名称以'i'开头，则为内部序列）添加到seq_type列表中
            
            if dist.shape[0] == 0:
                dist = np.array([float("inf")])
            
            for thr in rng: # 计算不同阈值下的平均误差和匹配数
                if seq_name[0] == 'i':
                    i_err[thr] += np.mean(dist <= thr)
                    i_matches[thr] += np.sum(dist <= thr)
                else:
                    v_err[thr] += np.mean(dist <= thr)
                    v_matches[thr] += np.sum(dist <= thr)
    
    seq_type = np.array(seq_type)
    n_feats = np.array(n_feats)
    n_matches = np.array(n_matches)
    
    return i_err, v_err, i_matches, v_matches, [seq_type, n_feats, n_matches]

In [10]:
def summary(stats):
    seq_type, n_feats, n_matches = stats
    print('# Features: {:f} - [{:d}, {:d}]'.format(np.mean(n_feats), np.min(n_feats), np.max(n_feats)))
    print('# Matches: Overall {:f}, Illumination {:f}, Viewpoint {:f}'.format(
        np.sum(n_matches) / ((n_i + n_v) * 5), 
        np.sum(n_matches[seq_type == 'i']) / (n_i * 5), 
        np.sum(n_matches[seq_type == 'v']) / (n_v * 5))
    )

In [11]:
def getBit(des):
    res = []
    for d in des:
        for i in range(8):
            res.append(((d >> i) & 1) * 2 - 1)
    return res

def generate_read_function(method, extension='ppm', type='float'): # 创建读取图像数据的函数
    def read_function(seq_name, im_idx):
        aux = np.load(os.path.join(dataset_path, seq_name, '%d.%s.%s' % (im_idx, extension, method))) # 从文件中加载数据
        if type == 'float': # 如果参数 type 为 'float'，则函数返回从加载的数据中得到的 'keypoints' 和 'descriptors'
            return aux['keypoints'], aux['descriptors']
        else: # 处理 'descriptors' 数据，通过解包其位并缩放值
            descriptors = np.unpackbits(aux['descriptors'], axis=1, bitorder='little')
            descriptors = descriptors * 2.0 - 1.0
            return aux['keypoints'], descriptors
    return read_function

In [12]:
def sift_to_rootsift(descriptors):
    return np.sqrt(descriptors / np.expand_dims(np.sum(np.abs(descriptors), axis=1), axis=1) + 1e-16)
def parse_mat(mat):
    keypoints = mat['keypoints'][:, : 2]
    raw_descriptors = mat['descriptors']
    l2_norm_descriptors = raw_descriptors / np.expand_dims(np.sum(raw_descriptors ** 2, axis=1), axis=1)
    descriptors = sift_to_rootsift(l2_norm_descriptors)
    return keypoints, descriptors

In [13]:
cache_dir = 'cache'
if not os.path.isdir(cache_dir):
    os.mkdir(cache_dir)

In [14]:
errors = {}

In [15]:
for method in methods:
    output_file = os.path.join(cache_dir, method + '.npy')
    print(method)
    if method == 'hesaff':
        read_function = lambda seq_name, im_idx: parse_mat(loadmat(os.path.join(dataset_path, seq_name, '%d.ppm.hesaff' % im_idx), appendmat=False))
    else:
        if method == 'delf' or method == 'delf-new':
            read_function = generate_read_function(method, extension='png')
        elif '+Boost-B' in method or (method.lower() == 'orb'):
            read_function = generate_read_function(method, type='binary')
        else:
            read_function = generate_read_function(method)  # 创建读取图像数据的函数read_function
    if os.path.exists(output_file):
        print('Loading precomputed errors...')
        errors[method] = np.load(output_file, allow_pickle=True)# 如果文件存在，使用 np.load 加载文件数据到 errors字典中xxxx
    else: # 主要在这里
        errors[method] = benchmark_features(read_function) # 如果不存在，调用 benchmark_features 函数对特征进行评估，并将结果存储在 errors 字典中
        np.save(output_file, errors[method])
    summary(errors[method][-1]) # 调用 summary 函数，传入 errors 字典中当前方法的最后一个元素，以生成和打印性能摘要

In [16]:
for method in methods:
    i_err, v_err, i_matches, v_matches, _ = errors[method]
    # 从 errors 字典中提取当前方法的错误和匹配信息，i_err 和 v_err 分别代表在不同阈值下的照明和视角错误，i_matches 和 v_matches 代表匹配的内点数
    print(method)
    for thr in [1, 3, 5]:
        print('# MMA@{:d}: Overall {:f}, Illumination {:f}, Viewpoint {:f}'.format( # 这里计算了整体的MMA、照明条件下的MMA和视角变化下的MMA
            thr,
            (i_err[thr] + v_err[thr]) / ((n_i + n_v) * 5), 
            i_err[thr] / (n_i * 5), 
            v_err[thr] / (n_v * 5))
        )
        print('# inliers@{:d}: Overall {:f}, Illumination {:f}, Viewpoint {:f}'.format( # 这里计算了整体的内点数、照明条件下的内点数和视角变化下的内点数
            thr,
            (i_matches[thr] + v_matches[thr]) / ((n_i + n_v) * 5), 
            i_matches[thr] / (n_i * 5), 
            v_matches[thr] / (n_v * 5))
        )

# Plotting

In [17]:
plt_lim = [1, 10]
plt_rng = np.arange(plt_lim[0], plt_lim[1] + 1)

In [18]:
plt.rc('axes', titlesize=25)
plt.rc('axes', labelsize=25)

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
for method, name, color, ls in zip(methods, names, colors, linestyles):
    i_err, v_err, _, _, _ = errors[method]
    plt.plot(plt_rng, [(i_err[thr] + v_err[thr]) / ((n_i + n_v) * 5) for thr in plt_rng], color=color, ls=ls, linewidth=2, label=name)
plt.title('Overall')
plt.xlim(plt_lim)
plt.xticks(plt_rng)
plt.ylabel('MMA')
plt.ylim([0, 1])
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=20)
plt.legend()

plt.subplot(1, 3, 2)
for method, name, color, ls in zip(methods, names, colors, linestyles):
    i_err, v_err, _, _, _ = errors[method]
    plt.plot(plt_rng, [i_err[thr] / (n_i * 5) for thr in plt_rng], color=color, ls=ls, linewidth=2, label=name)
plt.title('Illumination')
plt.xlabel('threshold [px]')
plt.xlim(plt_lim)
plt.xticks(plt_rng)
plt.ylim([0, 1])
plt.gca().axes.set_yticklabels([])
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=20)

plt.subplot(1, 3, 3)
for method, name, color, ls in zip(methods, names, colors, linestyles):
    i_err, v_err, _, _, _ = errors[method]
    plt.plot(plt_rng, [v_err[thr] / (n_v * 5) for thr in plt_rng], color=color, ls=ls, linewidth=2, label=name)
plt.title('Viewpoint')
plt.xlim(plt_lim)
plt.xticks(plt_rng)
plt.ylim([0, 1])
plt.gca().axes.set_yticklabels([])
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=20)

plt.savefig('hseq.pdf', bbox_inches='tight', dpi=300)

In [19]:
plt.rc('axes', titlesize=25)
plt.rc('axes', labelsize=25)

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
for method, name, color, ls in zip(methods, names, colors, linestyles):
    _, _, i_matches, v_matches, _ = errors[method]
    plt.plot(plt_rng, [(i_matches[thr] + v_matches[thr]) / ((n_i + n_v) * 5) for thr in plt_rng], color=color, ls=ls, linewidth=2, label=name)
plt.title('Overall')
plt.xlim(plt_lim)
plt.xticks(plt_rng)
plt.ylabel('Mean Inliners')
plt.ylim([0, 1500])
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=20)
plt.legend()

plt.subplot(1, 3, 2)
for method, name, color, ls in zip(methods, names, colors, linestyles):
    _, _, i_matches, v_matches, _ = errors[method]
    plt.plot(plt_rng, [i_matches[thr] / (n_i * 5) for thr in plt_rng], color=color, ls=ls, linewidth=2, label=name)
plt.title('Illumination')
plt.xlabel('threshold [px]')
plt.xlim(plt_lim)
plt.xticks(plt_rng)
plt.ylim([0, 1500])
plt.gca().axes.set_yticklabels([])
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=20)

plt.subplot(1, 3, 3)
for method, name, color, ls in zip(methods, names, colors, linestyles):
    _, _, i_matches, v_matches, _ = errors[method]
    plt.plot(plt_rng, [v_matches[thr] / (n_v * 5) for thr in plt_rng], color=color, ls=ls, linewidth=2, label=name)
plt.title('Viewpoint')
plt.xlim(plt_lim)
plt.xticks(plt_rng)
plt.ylim([0, 1500])
plt.gca().axes.set_yticklabels([])
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=20)